In [209]:
from collections import defaultdict
import pandas as pd

events = pd.read_csv('data/events.csv')
structure = pd.read_csv('data/structure.csv')

In [210]:
structure = structure.sort(['module_position', 'lesson_position', 'step_position'])
new_step_id = {}
old_step_id = {}
ind = 1
for x, row in structure.iterrows():
    cur_id = row['step_id']
    new_step_id[cur_id] = ind
    old_step_id[ind] = cur_id
    ind += 1

structure['step_id'] = structure['step_id'].apply(lambda x: new_step_id[x])
    
events['step_id'] = events['step_id'].apply(lambda x: new_step_id[x])
     


/home/pavel/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [211]:
data = pd.merge(events, structure, on='step_id')

In [212]:
data = data.sort(['step_id'])

/home/pavel/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [213]:
step_groupby = data.groupby('step_id')

In [214]:
user_groupby = data.groupby('user_id')

In [215]:
users_cnt = {}
for step_id, df in step_groupby:
    users_cnt[step_id] = len(df['user_id'].unique())

In [216]:
returned_cnt = defaultdict(int)
for user_id, df in user_groupby:
    cur_step_id_groupby = df.groupby('step_id')
    watch_time = {}
    for step_id, cdf in cur_step_id_groupby:
        watch_time[step_id] = sorted(list(cdf['time']))
    for step_id, times in watch_time.iteritems():
        first = times[0]
        last = times[-1]
        next_step = step_id + 1
        if next_step in watch_time:
            returned = False
            for time in watch_time[next_step]:
                if time > first and time < last:
                    returned = True
            if returned:
                returned_cnt[step_id] += 1
    

In [217]:
return_score = []
for step_id, cnt in returned_cnt.iteritems():
    score = float(cnt) / users_cnt[step_id]
    return_score.append((score, old_step_id[step_id]))

In [218]:
return_score = sorted(return_score, reverse=True)

In [219]:
','.join(map(lambda x: str(x[1]), return_score[:10]))

'41684,41604,41097,41481,42593,38872,41686,39735,39740,39717'